<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/qwen3_max_POC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://openrouter.ai/models

In [20]:
from openai import OpenAI
from google.colab import userdata

OPENROUTER_API_KEY=userdata.get('OPENROUTER_API_KEY')
if not OPENROUTER_API_KEY:
    raise ValueError("OPENROUTER_API_KEY is not set")

client = OpenAI(
  base_url="https://openrouter.ai/api/v1",
  api_key=OPENROUTER_API_KEY,
)

completion = client.chat.completions.create(
  extra_headers={
    "HTTP-Referer": "<YOUR_SITE_URL>", # Optional. Site URL for rankings on openrouter.ai.
    "X-Title": "<YOUR_SITE_NAME>", # Optional. Site title for rankings on openrouter.ai.
  },
  extra_body={},
  model="qwen/qwen3-max",
  messages=[
    {
      "role": "user",
      "content": "What is the meaning of Agentic AI?"
    }
  ]
)
print(completion.choices[0].message.content)

**Agentic AI** refers to artificial intelligence systems that exhibit **autonomous, goal-directed behavior**—acting as an "agent" that can perceive its environment, make decisions, and take actions to achieve specific objectives without constant human oversight.

Key characteristics of Agentic AI include:

1. **Autonomy**: The ability to operate independently, making decisions and taking actions based on its understanding of goals and context.
2. **Goal-Orientedness**: Designed to pursue specific objectives or tasks, often breaking them down into subtasks and planning accordingly.
3. **Reasoning and Planning**: Capable of strategic thinking—evaluating options, anticipating consequences, and adapting plans as needed.
4. **Reactivity and Adaptability**: Responds to changes in the environment or new information by adjusting its behavior.
5. **Proactivity**: Initiates actions to achieve goals rather than just reacting to inputs.

Agentic AI goes beyond passive tools (like a basic chatbot t

## Agentic Case

In [21]:
import json
from openai import OpenAI
from google.colab import userdata

# --- 1. SETUP AND AUTHENTICATION ---
# Securely retrieve the OpenRouter API Key from Google Colab secrets
OPENROUTER_API_KEY = userdata.get('OPENROUTER_API_KEY')
if not OPENROUTER_API_KEY:
    raise ValueError("OPENROUTER_API_KEY is not set in Colab secrets.")

# Initialize the OpenAI client pointing to the OpenRouter base URL
client = OpenAI(
  base_url="https://openrouter.ai/api/v1",
  api_key=OPENROUTER_API_KEY,
)

# --- 2. THE DIFFICULT AGENTIC TEST PROMPT ---

TEST_PROMPT = """
You are a Senior Data Engineer. Your task is to resolve a data pipeline discrepancy and present the finding as a structured JSON object. You must perform the following steps sequentially:

1.  **Logical Deduction (Constraint 1):** Five data streams (A, B, C, D, E) are received. B is only processed if A is present. C requires B to be processed first. D is independent. E is the final output layer and requires C and D to be complete. If Streams B and C both fail, the entire pipeline must report 'InconsistentState'. Which single stream's absence guarantees that 'InconsistentState' is the final required report, assuming all other streams are present and all processing steps except for C's dependency on B are successful?

2.  **Code Challenge (Constraint 2):** Based on the logical deduction in Step 1, write a short, self-contained Python function `check_inconsistency(streams_present)` that accepts a list of present streams (e.g., `['A', 'C', 'D', 'E']`) and returns the final required report string ('InconsistentState' or 'Success'). The function must explicitly check the condition where both B and C fail.

3.  **Meta-Reasoning & Output Formatting (Constraint 3):** Explain the reasoning for your Python function's logic, focusing on how the absence identified in Step 1 directly causes the B and C failure state. Finally, provide the entire output *only* as a single JSON object.

**REQUIRED JSON OUTPUT FORMAT (Strictly adhere to this structure):**
{
  "Analysis": "<Your full explanation from Step 3 here>",
  "PythonFunction": "<Your complete, triple-quoted Python function from Step 2 here>",
  "FinalReport": "<The single stream name (A, B, C, D, or E) from Step 1 here>"
}
"""

# --- 3. EXECUTE THE AGENTIC TEST ---
try:
    completion = client.chat.completions.create(
      # The Qwen3-Max model is selected
      model="qwen/qwen3-max",
      messages=[
        {"role": "user", "content": TEST_PROMPT}
      ],
      # Custom headers (OpenRouter specific)
      extra_headers={
        "HTTP-Referer": "https://colab.research.google.com",
        "X-Title": "Qwen3-Max Agent Test",
      },
      # Qwen3-specific parameter to trigger deep reasoning
      extra_body={
        "enable_thinking": True,    # Explicitly try to turn on Qwen's "Thinking Mode"
        "thinking_budget": 1024     # Allocate sufficient tokens for complex reasoning
      },
      # Ensure high creativity and thinking depth
      temperature=0.7,
      max_tokens=2048
    )

    # --- 4. PARSE AND DISPLAY RESULT ---

    raw_response = completion.choices[0].message.content
    print("--- RAW MODEL RESPONSE (EXPECTED JSON) ---")
    print(raw_response)

    # Attempt to parse the structured JSON output (Final test of constraint 3)
    try:
        # Find the start and end of the JSON object
        json_start = raw_response.find('{')
        json_end = raw_response.rfind('}')

        if json_start != -1 and json_end != -1:
            json_str = raw_response[json_start : json_end + 1]
            parsed_json = json.loads(json_str)

            print("\n--- PARSED & VALIDATED JSON OUTPUT ---")
            print(json.dumps(parsed_json, indent=4))

            # Final check of the key answer
            final_stream = parsed_json.get("FinalReport", "ERROR: Key not found")
            print(f"\n[Agentic Test Result]: The deduced critical stream is: {final_stream}")

        else:
            print("\n[Parsing Error]: The model failed to output a properly formatted JSON object.")

    except json.JSONDecodeError as e:
        print(f"\n[Parsing Error]: Failed to decode JSON. Model output was not valid JSON. Error: {e}")

except Exception as e:
    print(f"\n[API Error]: An error occurred during the API call: {e}")

--- RAW MODEL RESPONSE (EXPECTED JSON) ---
{
  "Analysis": "From the dependency rules: B requires A; C requires B; D is independent; E requires both C and D. The pipeline reports 'InconsistentState' only if both B and C fail. To guarantee this outcome, we must identify which single missing stream forces both B and C to fail, assuming all others are present and processing steps succeed except where dependencies block execution. If stream A is absent, then B cannot be processed (since B depends on A). Without B, C cannot be processed (since C depends on B). Thus, missing A causes both B and C to fail, triggering 'InconsistentState'. No other single missing stream guarantees both B and C fail: missing B directly causes C to fail (but B itself is missing, not failed due to dependency); however, the problem specifies that 'InconsistentState' is required only when both B and C fail, and the only way to guarantee that under the assumption that all other streams are present is by missing A—bec